# Map from geoJSON (covid data)
> Altair example combining data from Statistics Norway, NAV and Humdata

In [1]:
import deetly
import pandas as pd
import altair as alt
import dateutil.parser
import requests
import json
from pyjstat import pyjstat

from datetime import datetime
import time

'no_NO.UTF-8'

In [2]:
# Create datapackage
description = """
## Sykefraværstilfeller med Covid-19 diagnose - påvist eller mistanke om smitte


Sykefraværstilfellene i denne publiseringen bør tolkes med varsomhet. Statistikken er ikke å anse som offisiell statistikk fra NAV, men heller indikatorer som kan belyse effekten covid-19 har i samfunnet målt gjennom sykmeldinger fra helsevesenet. 

De fleste store EPJ-systemer (sykmeldernes elektroniske pasientjournalsystemer) er oppdatert med den nye koden for covid-19. Det tar noe tid før alle EPJ-systemer er oppdatert med kode for covid-19, og noe av økningen i sykmeldinger med covid-19 fremover kan derfor komme fra økt utbredelse på systemsiden.

I tillegg leverer sykehus sykmeldinger på papir, og disse fanges ikke opp elektronisk (papirsykmeldinger samles opp og lastes inn i datavarehus månedlig). I sum underrapporterer vi de faktiske forhold noe.

Les mer om den daglige statistikken over sykemeldinger med diagnose Covid-19 [her](https://www.nav.no/no/nav-og-samfunn/statistikk/flere-statistikkomrader/relatert-informasjon/sykefravaerstilfeller-pavist-eller-mistanke-om-smitte-av-koronaviruset/_/attachment/download/f6fafad7-fe0f-448f-a27f-8b6937a021f7:0b6147846fb34d40bbeb6d6764abb723d39b9f6a/Fakta%20om%20daglig%20sykmeldingsstatistikk%20over%20covid19.pdf)

Kilde: https://www.nav.no/no/nav-og-samfunn

"""

package = {
    "name":"Sykefraværstilfeller med Covid-19 diagnose", 
    "description":description, 
    "author":"Paul Bencze", 
    "theme":"example",
    "license": "MIT",
    "keywords": ["example, altair, covid-19, corona"]
}

dp = deetly.package(package)

Missing mandatory props(s): ['title']
Missing recommended props(s): ['contactPoint', 'distribution', 'keyword', 'publisher', 'spatial', 'temporal']
Property not in DCAT schema: ['name', 'author', 'license', 'keywords']
Invalid type: ['String:name', 'String:description', 'String:author', 'String or list of strings:theme', 'String:license', 'String:keywords', 'ISO date or datetime:issued', 'ISO date or datetime:modified']


## Get folketall pr fylke

In [3]:
query = {
  "query": [
    {
      "code": "Region",
      "selection": {
        "filter": "agg_single:Fylker2020",
        "values": [
          "30",
          "03",
          "34",
          "38",
          "42",
          "11",
          "46",
          "15",
          "50",
          "18",
          "54",
          "21"
        ]
      }
    },
    {
      "code": "Alder",
      "selection": {
        "filter": "agg:TodeltGrupperingB",
        "values": [
          "H17",
          "H18"
        ]
      }
    },
    {
      "code": "Tid",
      "selection": {
        "filter": "item",
        "values": [
          "2020"
        ]
      }
    }
  ],
  "response": {
    "format": "json-stat2"
  }
}


res = requests.post("https://data.ssb.no/api/v0/no/table/07459/", data=json.dumps(query))
dataset = pyjstat.Dataset.read(res.text)
df_befolkning = dataset.write('dataframe')
df_befolkning.head()

,region,alder,statistikkvariabel,år,value
0,Viken,0-17 år,Personer,2020,269533
1,Viken,18 år eller eldre,Personer,2020,971632
2,Oslo,0-17 år,Personer,2020,133128
3,Oslo,18 år eller eldre,Personer,2020,560366
4,Innlandet,0-17 år,Personer,2020,69410


In [4]:
df_befolkning_sum = df_befolkning[['region', 'statistikkvariabel','år','value']]\
    .groupby(['region', 'statistikkvariabel','år']).sum().reset_index()

In [5]:
df_befolkning_sum['region'] = df_befolkning_sum['region'].apply(lambda x: x.split('-')[0].strip())

In [6]:
df_befolkning_sum['region'].unique()

array(['Agder', 'Innlandet', 'Møre og Romsdal', 'Nordland', 'Oslo',
       'Rogaland', 'Svalbard', 'Troms og Finnmark', 'Trøndelag',
       'Vestfold og Telemark', 'Vestland', 'Viken'], dtype=object)

In [7]:
df_befolkning_sum

,region,statistikkvariabel,år,value
0,Agder,Personer,2020,307231
1,Innlandet,Personer,2020,371385
2,Møre og Romsdal,Personer,2020,265238
3,Nordland,Personer,2020,241235
4,Oslo,Personer,2020,693494
5,Rogaland,Personer,2020,479892
6,Svalbard,Personer,2020,0
7,Troms og Finnmark,Personer,2020,243311
8,Trøndelag,Personer,2020,468702
9,Vestfold og Telemark,Personer,2020,419396


In [8]:
dataset = pyjstat.Dataset.read(res.text)
df_ssb = dataset.write('dataframe')
df_ssb.head()

,region,alder,statistikkvariabel,år,value
0,Viken,0-17 år,Personer,2020,269533
1,Viken,18 år eller eldre,Personer,2020,971632
2,Oslo,0-17 år,Personer,2020,133128
3,Oslo,18 år eller eldre,Personer,2020,560366
4,Innlandet,0-17 år,Personer,2020,69410


## Get data from nav.no

In [9]:
EXCEL_URL_SYKEFRAVAER = 'https://www.nav.no/no/nav-og-samfunn/statistikk/flere-statistikkomrader/relatert-informasjon/sykefravaerstilfeller-pavist-eller-mistanke-om-smitte-av-koronaviruset/_/attachment/download/cec6b8f2-2906-4fc0-a51b-111f15acd7cc:dc45e15a9ab040490ceb74a829ccff757fe5b048/0804%20-%20Sykefrav%C3%A6rstilfeller%20-%20corona%20(dag%20og%20uke).xlsx'

In [10]:
MAP_URL = 'https://raw.githubusercontent.com/datasett/maps/master/norway/data/2020/fylker_2020_s_geojson.json'

In [11]:
xl_sykefravaer = pd.ExcelFile(EXCEL_URL_SYKEFRAVAER)
xl_sykefravaer.sheet_names

['Per dag',
 'Fylke dag',
 'Fylke uke',
 'Fylke andel totalt',
 'Kommune antall og andel',
 'Bydel antall og andel',
 'Utvikling i antall tilfeller',
 'Om statistikken']

In [12]:
df = pd.read_excel(EXCEL_URL_SYKEFRAVAER,sheet_name='Fylke dag', header=8)
df.drop(columns=["Unnamed: 0","Unnamed: 15"], axis=1, inplace=True)
df.rename(columns={"Unnamed: 1": "Time"}, inplace=True)
df.drop([0, 0],inplace=True)
df.head(1)

,Time,Hele landet,Oslo,Rogaland,Møre og Romsdal,Nordland,Viken,Innlandet,Vestfold og Telemark,Agder,Vestland,Trøndelag,Troms og Finnmark,Ukjent
1,2020-04-07 00:00:00,221,63,10,*,*,72,8,17,14,14,13,*,*


In [13]:
source = df[['Time', 'Hele landet']].copy()
source['Dag'] = source['Time'].apply(lambda x: str(x.strftime("%d %B %Y")))
source['Dag Navn'] = source['Time'].apply(lambda x: str(x.strftime("%A")))
source['Time'] = pd.to_datetime(source['Time'])
source = source.sort_values(by='Time', ascending='False')
source['Sum'] = source['Hele landet'].cumsum()
source = source.rename(columns={'Hele landet': 'Nye', 'Time': 'Dato'})
source.head(1)

,Dato,Nye,Dag,Dag Navn,Sum
31,2020-03-08,124,08 mars 2020,søndag,124


In [14]:
base = alt.Chart(source).encode(
    x=alt.X('Dato:T', axis=alt.Axis(title='', format="%d.%m.%Y", labelAngle=90, grid=False))
)

bar = base.mark_bar(color='rgba(124,194,189,1)').encode(
    y=alt.Y('Nye:Q', axis=alt.Axis(title='')),
    tooltip =[alt.Tooltip('Dag Navn:O', title='Dag'), alt.Tooltip('Dag:O', title='Dato'), alt.Tooltip('Nye:Q', title='Antall nye')],
)

line = base.mark_line(color='rgba(124,194,189,1)').encode(
    y='Sum:Q'
)

fig = (bar+line)
fig

alt.LayerChart(...)

In [15]:
total = source['Nye'].sum()
last_date = source['Dato'].max().strftime("%d %B %Y")
title = f"Sykefraværstilfeller"
description = f"Antall sykefraværstilfeller registert med startdato. Totalt antall pr {last_date}: {total}"
dp.vega(fig,title, description)

## Get data from humdata

In [16]:
DATA_URL_CONFIRMED= 'https://data.humdata.org/hxlproxy/api/data-preview.csv?url=https%3A%2F%2Fraw.githubusercontent.com%2FCSSEGISandData%2FCOVID-19%2Fmaster%2Fcsse_covid_19_data%2Fcsse_covid_19_time_series%2Ftime_series_covid19_confirmed_global.csv&filename=time_series_covid19_confirmed_global.csv'

In [17]:
df_confirmed = pd.read_csv(DATA_URL_CONFIRMED)

In [18]:
selection = df_confirmed[ df_confirmed['Country/Region'].isin(['Norway'])].copy()
selection.drop(columns=['Province/State','Lat','Long'], inplace=True)
selection['Source'] = 'Humdata'

In [19]:
source = pd.melt(selection, id_vars=['Source', 'Country/Region'])
source['change'] = source['value']- source['value'].shift(1)
source['date'] = source['variable'].apply(lambda x: dateutil.parser.parse(x))

In [20]:
first = max(source[source['value']==0]['date'])
first

Timestamp('2020-02-25 00:00:00')

In [21]:
source = source[source['date'] >= first]

In [22]:
base = alt.Chart(source).encode(
    x=alt.X('date:T', axis=alt.Axis(title='', format="%d %b %Y", labelAngle=90, grid=False))
)

bar = base.mark_bar(color='rgba(124,194,189,1)').encode(
    y=alt.Y('change:Q', axis=alt.Axis(title='')),
    tooltip =[alt.Tooltip('variable:O', title='Dato'), alt.Tooltip('change:Q', title='Antall')],
)

line = base.mark_line(color='rgba(124,194,189,1)').encode(
    y='value:Q',
    tooltip =[alt.Tooltip('variable:O', title='Dato'), alt.Tooltip('value:Q', title='Antall')],
)

fig = (bar+line).interactive()

fig

alt.LayerChart(...)

In [23]:
source.head(1)

,Source,Country/Region,variable,value,change,date
34,Humdata,Norway,2/25/20,0,0.0,2020-02-25


In [24]:
total = int(source['change'].sum())
last_date = source['date'].max().strftime("%d %B %Y")
title = f"Bekreftet smittet"
description = f"Antall bekreftet smittet. Totalt antall pr {last_date}: {total}.  Kilde: [The Humanitarian Data Exchange](https://data.humdata.org)"
dp.vega(fig,title, description)

In [25]:
from datetime import datetime
dt = datetime.combine(datetime.now().date(), datetime.min.time())
today = int(time.mktime(dt.timetuple()))

## Display map

In [26]:
data = pd.melt(df, id_vars=['Time'])
data['day_str'] = data['Time'].apply(lambda x: str(x.strftime("%d %B %Y")))
data['day_of_week'] = data['Time'].apply(lambda x: str(x.strftime("%A")))
data.columns=['time','name','antall','day_str','day_of_week']
data['day'] = data['time'].apply(lambda x: int(int(today - int(time.mktime(x.timetuple())))/86400))
data['name'] = data['name'].apply(lambda x: str(x).strip())
data['fylke_navn'] = data['name'].apply(lambda x: str(x).upper().strip())
data = data[~data['fylke_navn'].isin(['HELE LANDET', 'UKJENT'])]
data['antall']= data['antall'].replace('*', 0) 
data.head(1)

,time,name,antall,day_str,day_of_week,day,fylke_navn
31,2020-04-07,Oslo,63,07 april 2020,tirsdag,200,OSLO


In [27]:
df_merged = pd.merge(data, df_befolkning_sum, how='left', left_on='name', right_on='region')

In [28]:
df_merged['region'].unique()

array(['Oslo', 'Rogaland', 'Møre og Romsdal', 'Nordland', 'Viken',
       'Innlandet', 'Vestfold og Telemark', 'Agder', 'Vestland',
       'Trøndelag', 'Troms og Finnmark'], dtype=object)

In [29]:
data['name'].unique()

array(['Oslo', 'Rogaland', 'Møre og Romsdal', 'Nordland', 'Viken',
       'Innlandet', 'Vestfold og Telemark', 'Agder', 'Vestland',
       'Trøndelag', 'Troms og Finnmark'], dtype=object)

In [30]:
df_merged['andel'] = 1000 * (df_merged['antall']/df_merged['value'])

In [31]:
source = df_merged.sort_values(['name','time'])
source['sum_antall'] = source.groupby('name')['antall'].transform(pd.Series.cumsum)
source['sum_andel'] = source.groupby('name')['andel'].transform(pd.Series.cumsum)
source.head(1)

,time,name,antall,day_str,day_of_week,day,fylke_navn,region,statistikkvariabel,år,value,andel,sum_antall,sum_andel
247,2020-03-08,Agder,0,08 mars 2020,søndag,229,AGDER,Agder,Personer,2020,307231,0.0,0,0.0


## Nye sykefraværstilfeller

In [32]:
def getMapNewCases(day, title=""):
    
    selected = source[source['day']==day]
    day_str = f"{title} {selected['day_str'].iloc[0]}"
    
    values = alt.pipe(selected, alt.to_values)
    
    map_data = alt.Data(url=MAP_URL, format=alt.DataFormat(property='features',type='json'))

    fig = alt.Chart(map_data).mark_geoshape(
    ).encode(
        color = alt.Color('antall:Q'),
        tooltip =[alt.Tooltip('day_of_week:N', title='Dag'), alt.Tooltip('properties.fylke_navn:N', title='Fylke'),alt.Tooltip('antall:Q', title='Antall')],
    ).transform_lookup(
        lookup='properties.fylke_navn',
        from_=alt.LookupData(values, 'fylke_navn', ['antall', 'day_str','day_of_week'])
    ).properties(
        projection={'type': 'mercator'}
    ).properties(
        title=day_str
    )
    
    return fig

In [33]:
first_day = int(source['day'].min())
last_day = int(source['day'].max())
print(first_day, last_day)

200 229


In [34]:
fig = getMapNewCases(first_day)
for day in range(first_day+1,last_day):
    fig = fig | getMapNewCases(day)
    
fig.configure_legend(
  orient='top'
).configure_view(
    strokeWidth=0
)

/Users/B149899/.pyenv/versions/3.8.2/envs/pydeck/lib/python3.8/site-packages/altair/utils/data.py:226: AltairDeprecationWarning:

alt.pipe() is deprecated, and will be removed in a future release. Use toolz.curried.pipe() instead.



alt.HConcatChart(...)

In [35]:
description = f"Antall nye sykefraværstilfeller"
title = f"Antall nye"
dp.vega(fig,title, description)

## Nye sykefraværtilfeller akkumulert antall

In [36]:
def getMapNewCasesSum(day, title=""):
    
    selected = source[source['day']==day]
    day_str = f"{title} {selected['day_str'].iloc[0]}"
    
    values = alt.pipe(selected, alt.to_values)
    
    map_data = alt.Data(url=MAP_URL, format=alt.DataFormat(property='features',type='json'))

    fig = alt.Chart(map_data).mark_geoshape(
    ).encode(
        color = alt.Color('sum_antall:Q'),
        tooltip =[alt.Tooltip('day_of_week:N', title='Dag'), alt.Tooltip('properties.fylke_navn:N', title='Fylke'),alt.Tooltip('sum_antall:Q', title='Antall')],
    ).transform_lookup(
        lookup='properties.fylke_navn',
        from_=alt.LookupData(values, 'fylke_navn', ['sum_antall', 'day_str','day_of_week'])
    ).properties(
        projection={'type': 'mercator'}
    ).properties(
        title=day_str
    )
    
    return fig

In [37]:
fig = getMapNewCasesSum(first_day)
for day in range(first_day+1,last_day):
    fig = fig | getMapNewCasesSum(day)
    
fig.configure_legend(
  orient='top'
).configure_view(
    strokeWidth=0
)

alt.HConcatChart(...)

In [38]:
description = f"Akkumulert antall sykefraværstilfeller"
title = f"Akkumulert antall"
dp.vega(fig,title, description)

## Nye sykefraværtilfeller som andel av befolkning 

In [39]:
def getMapCasesPerHabitant(day,title=""):
    
    selected = source[source['day']==day]
    day_str = f"{title} {selected['day_str'].iloc[0]}"
    
    values = alt.pipe(selected, alt.to_values)
    
    map_data = alt.Data(url=MAP_URL, format=alt.DataFormat(property='features',type='json'))

    fig = alt.Chart(map_data).mark_geoshape(
    ).encode(
        color = alt.Color('andel:Q'),
        tooltip =[alt.Tooltip('day_of_week:N', title='Dag'), alt.Tooltip('properties.fylke_navn:N', title='Fylke'),alt.Tooltip('andel:Q', title='Nye tilfeller / 1000 inbyggere', format=",.2f")],
    ).transform_lookup(
        lookup='properties.fylke_navn',
        from_=alt.LookupData(values, 'fylke_navn', ['andel', 'day_str','day_of_week'])
    ).properties(
        projection={'type': 'mercator'}
    ).properties(
        title=day_str
    )
    
    return fig

In [40]:
fig = getMapCasesPerHabitant(first_day)
for day in range(first_day+1,last_day):
    fig = fig | getMapCasesPerHabitant(day)
    
fig.configure_legend(
  orient='top'
).configure_view(
    strokeWidth=0
)

alt.HConcatChart(...)

In [41]:
description = f"Antall pr 1000 innbyggere"
title = f"Antall pr 1000 innbyggere"
dp.vega(fig,title, description)

## Nye sykefraværstilfeller akkumulert pr 1000 innbyggere

In [42]:
def getMapCasesPerHabitantSum(day, title=""):
    
    selected = source[source['day']==day]
    day_str = f"{title} {selected['day_str'].iloc[0]}"
    
    values = alt.pipe(selected, alt.to_values)
    
    map_data = alt.Data(url=MAP_URL, format=alt.DataFormat(property='features',type='json'))

    fig = alt.Chart(map_data).mark_geoshape(
    ).encode(
        color = alt.Color('sum_andel:Q'),
        tooltip =[alt.Tooltip('day_of_week:N', title='Dag'), alt.Tooltip('properties.fylke_navn:N', title='Fylke'),alt.Tooltip('sum_andel:Q', title='Akkumulert / 1000 inbyggere', format=",.2f")],
    ).transform_lookup(
        lookup='properties.fylke_navn',
        from_=alt.LookupData(values, 'fylke_navn', ['sum_andel', 'day_str','day_of_week'])
    ).properties(
        projection={'type': 'mercator'}
    ).properties(
        title=day_str
    )
    
    return fig

In [43]:
fig = getMapCasesPerHabitantSum(first_day)
for day in range(first_day+1,last_day):
    fig = fig | getMapCasesPerHabitantSum(day)
    
fig.configure_legend(
  orient='top'
).configure_view(
    strokeWidth=0
)

alt.HConcatChart(...)

In [44]:
description = f"Akkumulert antall pr 1000 innbyggere"
title = f"Akkumulert antall pr 1000 innbyggere"
dp.vega(fig,title, description)

## Intermixed

In [45]:
fig = getMapCasesPerHabitant(first_day, "Antall nye registrert pr 1000 personer")
fig = fig | getMapCasesPerHabitantSum(first_day, "Totalt antall pr 1000 personer")
for day in range(first_day+1,last_day):
    fig = alt.vconcat(fig, getMapCasesPerHabitant(day, "Antall nye registrert pr 1000 personer") | getMapCasesPerHabitantSum(day, "Totalt antall pr 1000 personer"))
    
fig.configure_legend(
  orient='top'
).configure_view(
    strokeWidth=0
).configure_concat(
    spacing=100
)

alt.VConcatChart(...)

In [46]:
description = f"Antall og akkumulert antall pr 1000 innbyggere"
title = f"Antall og akkumulert antall pr 1000 innbyggere"
dp.vega(fig,title, description)

## Publish

In [47]:
item = dp.publish()

View: https://public.deetly.com/examples/c73752be0cd93033e3c8d0634a92b669 

Metadata: https://storage.googleapis.com/deetly/examples/c73752be0cd93033e3c8d0634a92b669/datapackage.json 

